In [1]:

Data=[[j for j in i.replace("\n","").split(" ") if j!='' and not j.strip().isnumeric()] for i in open("../data.txt",encoding="utf-8")]
freq={}

for i in Data:
    
    for j in i:
        if j in freq:
            freq[j]+=1
        else:
            freq[j]=1


In [2]:
l=len(Data)
min_support = 100/l
words=set(i for i in freq.keys() if freq[i]/l>=min_support)
if 'based' in words:
    words.remove('based')
data=[sorted(set(j for j in i if j in words),reverse=True,key=lambda k:freq[k]) for i in Data]
data=[i for i in data if len(i)!=0]

In [3]:
from collections import defaultdict as dt
class node:
    def __init__(self,v=None):
        self.val=v
        self.child=dt(node)
        self.count=0

In [4]:
def tree(root,x,i):
    if i>=len(x):
        return
    zz=root.child[x[i]]
    zz.count+=1
    zz.val=x[i]
    tree(zz,x,i+1)

In [5]:
root=node()
for i in data:
    tree(root,i,0)

In [16]:

def traverse(root,z):
    res=dt(int)
    wd=set()
    for a in root.child:
        z.append(a)
        qq=root.child[a]
        di=traverse(qq,z)
        del z[-1]
        if not z:continue
        for b in di:
            res[b]+=di[b]
            wd.add(b)       
    if not z:return
    for i in wd:
        if res[i]/l>=min_support:
            patterns[i].append((tuple(z),res[i]))
            res[i]=0
    if root.val!=None:
        res[root.val]+=root.count
    return res
        
        

In [17]:
patterns=dt(list)
traverse(root,[])
print(len(patterns))

3629


In [18]:
ind=dt(list)
new_words=set()
for i in patterns:
    new_words.add(i)
    for j,k in patterns[i]:
        new_words=new_words.union(j)
print(len(new_words))
d=0

for j in range(len(data)):
    for i in data[j]:
        if i in new_words:
            ind[i].append(j)

3633


In [19]:
def support(x):
    word = min((len(ind[i]),i) for i in x)[1]
    c=0
    for j in ind[word]:
        flag=False
        for i in x:
            if i not in data[j]:
                flag=True
                break
        if not flag:
            c+=1
    return c

In [22]:
res=set()
def func(a,b,i,x,r):
    if i>=len(x):
        if a and b:
            if len(a)==1:
                supp_a=freq[list(a)[0]]
            else:
                supp_a=support(a)
            if supp_a==0:return
            conf = support(a.union(b))/supp_a
            if conf>1:
                print(x,a)
            if conf>=0.7:
                s="("
                for j in sorted(a):
                    s+=j+" ; "
                s+="), ("
                for j in sorted(b):
                    s+=j+" ; "
                s+="), "+str(conf)+"\n"
                res.add(s)
        return
    a.add(x[i])
    func(a,b,i+1,x,r)
    a.remove(x[i])
    b.add(x[i])
    func(a,b,i+1,x,r)
    b.remove(x[i])
    func(a,b,i+1,x,r)

In [23]:
d=0
for b in patterns:
    for a,c in patterns[b]:
        func(set(),{b},0,list(a),c)
        func({b},set(),0,list(a),c)
    d+=1
    print(d,end='\r')
        

    

In [24]:
file=open("association_rules.csv",'w',encoding="utf-8")
file.write("antecedent,consequent,confidence\n")
for i in res:
    file.write(i)
file.close()